<a href="https://colab.research.google.com/github/azaslight/Skin-Cancer-Classifier-modelathon/blob/main/area51_telegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Убедись, что telebot установлен
!pip install pyTelegramBotAPI

# Импорты
import telebot
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
import joblib

# Инициализация Telegram-бота
API_TOKEN = '7107456211:AAESH3zPCwzgakOyxFTx6m50esN6XXdgiA4'
bot = telebot.TeleBot(API_TOKEN)

# Загрузка модели и label_encoder
model = load_model('skin_cancer_classifier.h5')
label_encoder = joblib.load('label_encoder.pkl')  # Загрузка label_encoder
CLASS_NAMES = label_encoder.classes_              # Определение классов
print(f"Загружены классы: {CLASS_NAMES}")

IMG_SIZE = 64

def predict_image(img_path):
    img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    predicted_class = CLASS_NAMES[np.argmax(prediction)]
    confidence = np.max(prediction)
    return predicted_class, confidence

@bot.message_handler(commands=['start'])
def welcome(message):
    bot.send_message(message.chat.id, "Добро пожаловать! Отправьте мне изображение кожного поражения, и я попытаюсь классифицировать его.")

@bot.message_handler(content_types=['photo'])
def handle_image(message):
    photo = message.photo[-1]
    file_id = photo.file_id
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    img_path = 'received_image.jpg'
    with open(img_path, 'wb') as new_file:
        new_file.write(downloaded_file)

    predicted_class, confidence = predict_image(img_path)
    bot.reply_to(message, f"Результат: {predicted_class}\nУверенность: {confidence * 100:.2f}%")

# Запуск бота
bot.polling()

Загружены классы: ['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
